#Genre detection

In [74]:
pip install transformers

In [75]:
from google.colab import files
import pandas as pd
data_to_load = files.upload()
import io

# Replace 'path_to_your_file' with the path to the dataset file you uploaded
#df  = pd.read_excel(io.BytesIO(data_to_load['labeled_dataset (2).xlsx']))
# df = pd.read_csv('trimmed_dataset')

# Load the dataset
df = pd.read_csv('ajibawa_dataset (3).csv')
df.head()

Saving ajibawa_dataset.csv to ajibawa_dataset (3).csv


,text_token_length,text,prompt
0,164,"In a small town named Harmonyville, there liv...",Write an educational story (3-5 paragraphs) ta...
1,161,"Once upon a time, in a small town named Harmo...",Write an educational story (3-5 paragraphs) ta...
2,183,In a small town by the sea lived two best fri...,Write an educational story (3-5 paragraphs) ta...
3,191,Sally loved to read books. She would spend ho...,Write an educational story (3-5 paragraphs) ta...
4,154,Little Gracie loved visiting the town library...,Write an educational story (3-5 paragraphs) ta...


In [76]:
df = df.head(150)

In [77]:
df.shape

(150, 3)

In [78]:
import pandas as pd
from transformers import pipeline

# Initialize the zero-shot classification pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Define your function for classification
def classify_text(text, candidate_genres=["health", "sport", "adventure", "family", "friendship", "animals", "books", "Kindness", "Courage", "Curiosity and Learning", "Responsibility"]):
    try:
        result = classifier(text, candidate_genres)
        return result['labels'][0]
    except Exception as e:
        return "Error"

# Apply the function to your DataFrame (consider doing this in batches for large datasets)
df['predicted_genre'] = df['text'].apply(classify_text)

# Save the modified DataFrame back to a CSV file
#df.to_csv('path_to_your_modified_dataset.csv', index=False)


In [79]:
df

,text_token_length,text,prompt,predicted_genre
0,164,"In a small town named Harmonyville, there liv...",Write an educational story (3-5 paragraphs) ta...,adventure
1,161,"Once upon a time, in a small town named Harmo...",Write an educational story (3-5 paragraphs) ta...,adventure
2,183,In a small town by the sea lived two best fri...,Write an educational story (3-5 paragraphs) ta...,Responsibility
3,191,Sally loved to read books. She would spend ho...,Write an educational story (3-5 paragraphs) ta...,books
4,154,Little Gracie loved visiting the town library...,Write an educational story (3-5 paragraphs) ta...,Curiosity and Learning
...,...,...,...,...
145,206,Ben was feeling hungry after a long day of pl...,Write an educational story (3-5 paragraphs) ta...,Curiosity and Learning
146,154,"One sunny day, Ali the Ant and his friend Pol...",Write an educational story (3-5 paragraphs) ta...,Curiosity and Learning
147,189,Little Violet was having trouble solving her ...,Write an educational story (3-5 paragraphs) ta...,adventure
148,160,"It was a sunny Sunday afternoon, and best fri...",Write an educational story (3-5 paragraphs) ta...,friendship


#Sentiment recognition

In [80]:
!pip install nltk

In [81]:
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from transformers import pipeline
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [82]:
!pip install dask[dataframe] --quiet

In [83]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()

def get_vader_sentiment(text):
    score = sia.polarity_scores(text)
    if score['compound'] >= 0.05:
        return 'positive'
    elif score['compound'] <= -0.05:
        return 'negative'
    else:
        return 'neutral'


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [84]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd

# Assuming 'stories_df' is your DataFrame and the texts are in the 'text' column

def apply_sentiment_parallel(df, function, column_name):
    with ThreadPoolExecutor() as executor:
        # Creating a future for each apply operation
        futures = {executor.submit(function, text): text for text in df['text']}
        for future in as_completed(futures):
            sentiment = future.result()
            df.at[futures[future], column_name] = sentiment
    return df


df['vader_sentiment'] = ''
df = apply_sentiment_parallel(df, get_vader_sentiment, 'vader_sentiment')

In [85]:
pip install pandas vaderSentiment

In [86]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer  # Ensure VADER is installed

def get_vader_sentiment(text):
    """
    Analyzes the sentiment of a given text using VADER sentiment analysis.

    Parameters:
    - text (str): The text to analyze.

    Returns:
    - str: The sentiment classification result ('Positive', 'Neutral', or 'Negative').
    """
    analyzer = SentimentIntensityAnalyzer()
    sentiment_score = analyzer.polarity_scores(text)
    compound_score = sentiment_score['compound']
    if compound_score >= 0.05:
        return 'Positive'
    elif compound_score <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'

def apply_sentiment_parallel(df, function, column_name='text'):
    """
    Applies a given function in parallel to a specified column in a DataFrame and
    stores the result in a new column.

    Parameters:
    - df (pd.DataFrame): The DataFrame to process.
    - function (callable): The function to apply to each element in the column.
    - column_name (str): The name of the column to process.

    Returns:
    - pd.DataFrame: The DataFrame with the results of the function application.
    """
    with ThreadPoolExecutor() as executor:
        # Convert each entry to string and submit for processing, tracking by row index
        futures = {executor.submit(function, str(df.at[index, 'text'])): index for index in df.index}

        # Collect and assign the results as they complete
        for future in as_completed(futures):
            sentiment = future.result()
            df.at[futures[future], 'vader_sentiment'] = sentiment
    return df

# Assuming 'df' is your DataFrame and it includes a 'text' column
# Example DataFrame initialization (replace with your actual DataFrame loading)
# df = pd.DataFrame({'text': ["I love this!", "This is bad."]})
# Ensure a column for the sentiment results
df['vader_sentiment'] = ''

# Apply the sentiment analysis in parallel
df = apply_sentiment_parallel(df, get_vader_sentiment, 'text')

# Example to display the DataFrame
print(df)


                                                    text_token_length  \
0                                                               164.0   
1                                                               161.0   
2                                                               183.0   
3                                                               191.0   
4                                                               154.0   
...                                                               ...   
 It was a sunny Sunday afternoon, and best frie...                NaN   
 In a bustling town filled with curious creatur...                NaN   
 Ben was feeling hungry after a long day of pla...                NaN   
 Once upon a time, in a land far away, there we...                NaN   
 Once upon a time, in a small town named Harmon...                NaN   

                                                                                                 text  \
0                 

In [87]:
df = df.head(150)

In [88]:
df

,text_token_length,text,prompt,predicted_genre,vader_sentiment
0,164.0,"In a small town named Harmonyville, there liv...",Write an educational story (3-5 paragraphs) ta...,adventure,Positive
1,161.0,"Once upon a time, in a small town named Harmo...",Write an educational story (3-5 paragraphs) ta...,adventure,Positive
2,183.0,In a small town by the sea lived two best fri...,Write an educational story (3-5 paragraphs) ta...,Responsibility,Positive
3,191.0,Sally loved to read books. She would spend ho...,Write an educational story (3-5 paragraphs) ta...,books,Positive
4,154.0,Little Gracie loved visiting the town library...,Write an educational story (3-5 paragraphs) ta...,Curiosity and Learning,Positive
...,...,...,...,...,...
145,206.0,Ben was feeling hungry after a long day of pl...,Write an educational story (3-5 paragraphs) ta...,Curiosity and Learning,Positive
146,154.0,"One sunny day, Ali the Ant and his friend Pol...",Write an educational story (3-5 paragraphs) ta...,Curiosity and Learning,Positive
147,189.0,Little Violet was having trouble solving her ...,Write an educational story (3-5 paragraphs) ta...,adventure,Positive
148,160.0,"It was a sunny Sunday afternoon, and best fri...",Write an educational story (3-5 paragraphs) ta...,friendship,Positive


In [89]:
number_of_values_in_column_A = df['vader_sentiment'].value_counts()
print(f"Unique values in column 'A': {number_of_values_in_column_A}")

Unique values in column 'A': vader_sentiment
Positive    147
Negative      3
Name: count, dtype: int64


#Entity recognition

In [90]:
!pip install spacy
!python -m spacy download en_core_web_sm

import spacy

# Load your SpaCy model
nlp = spacy.load("en_core_web_sm")

# Function to process a chunk and extract personas and cities
def process_chunk(chunk):
    personas = []
    cities = []
    for doc in nlp.pipe(chunk):
        personas.extend([ent.text for ent in doc.ents if ent.label_ == "PERSON"])
        cities.extend([ent.text for ent in doc.ents if ent.label_ == "GPE"])
    return personas, cities


# Function to split text into chunks
def chunk_text(text, chunk_size):
    chunks = []
    for i in range(0, len(text), chunk_size):
        chunks.append(text[i:i + chunk_size])
    return chunks

# Function to extract entities from text
def extract_entities(text):
    chunk_size = 200  # Adjust the chunk size as needed
    chunks = chunk_text(text, chunk_size)

    personas = []
    cities = []

    for chunk in chunks:
        chunk_personas, chunk_cities = process_chunk([chunk])
        personas.extend(chunk_personas)
        cities.extend(chunk_cities)

    return ", ".join(personas), ", ".join(cities)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 54.9 MB/s eta 0:00:00

Aborted.


/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [91]:
import pandas as pd
import spacy

# Load your SpaCy model
nlp = spacy.load("en_core_web_sm")

# Function to process a chunk and extract personas and cities
def process_chunk(chunk):
    personas = set()  # Use a set to store unique personas
    cities = set()  # Use a set to store unique cities
    for doc in nlp.pipe(chunk):
        personas.update(ent.text for ent in doc.ents if ent.label_ == "PERSON")
        cities.update(ent.text for ent in doc.ents if ent.label_ == "GPE")
    return personas, cities

# Function to split text into chunks
def chunk_text(text, chunk_size):
    chunks = []
    for i in range(0, len(text), chunk_size):
        chunks.append(text[i:i + chunk_size])
    return chunks

# Function to extract entities from text
def extract_entities(text):
    chunk_size = 200  # Adjust the chunk size as needed
    chunks = chunk_text(text, chunk_size)

    personas = set()  # Initialize an empty set for personas
    cities = set()  # Initialize an empty set for cities

    for chunk in chunks:
        chunk_personas, chunk_cities = process_chunk([chunk])
        personas.update(chunk_personas)
        cities.update(chunk_cities)

    return ", ".join(sorted(personas)), ", ".join(sorted(cities))

# Function to apply entity extraction to a DataFrame column
def apply_entity_extraction_to_dataframe(df, column_name):
    # Apply the extract_entities function to each row in the specified column
    extracted_entities = df[column_name].apply(lambda x: extract_entities(x))

    # Split the tuples returned by extract_entities into separate lists
    personas, cities = zip(*extracted_entities)

    # Add the extracted personas and cities as new columns in the DataFrame
    df['personas'] = personas
    df['cities'] = cities

    return df

# Apply the function to your DataFrame
df = apply_entity_extraction_to_dataframe(df, 'text')

<ipython-input-91-824b7bedbfb8>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['personas'] = personas
<ipython-input-91-824b7bedbfb8>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cities'] = cities


In [92]:
df

,text_token_length,text,prompt,predicted_genre,vader_sentiment,personas,cities
0,164.0,"In a small town named Harmonyville, there liv...",Write an educational story (3-5 paragraphs) ta...,adventure,Positive,"Alex, Avril Lavigne, Harmonyville, John Wooden...",
1,161.0,"Once upon a time, in a small town named Harmo...",Write an educational story (3-5 paragraphs) ta...,adventure,Positive,"Sam, Science",Harmonyville
2,183.0,In a small town by the sea lived two best fri...,Write an educational story (3-5 paragraphs) ta...,Responsibility,Positive,"Alex, Green, Sam",
3,191.0,Sally loved to read books. She would spend ho...,Write an educational story (3-5 paragraphs) ta...,books,Positive,"Ben, Sally",
4,154.0,Little Gracie loved visiting the town library...,Write an educational story (3-5 paragraphs) ta...,Curiosity and Learning,Positive,"Gracie, Mabel",
...,...,...,...,...,...,...,...
145,206.0,Ben was feeling hungry after a long day of pl...,Write an educational story (3-5 paragraphs) ta...,Curiosity and Learning,Positive,"Ben, Sarah",
146,154.0,"One sunny day, Ali the Ant and his friend Pol...",Write an educational story (3-5 paragraphs) ta...,Curiosity and Learning,Positive,"Ali, Ali the, Pola Penguin, Zara, Zara Zebra","Farsi, Iran, Poland"
147,189.0,Little Violet was having trouble solving her ...,Write an educational story (3-5 paragraphs) ta...,adventure,Positive,,
148,160.0,"It was a sunny Sunday afternoon, and best fri...",Write an educational story (3-5 paragraphs) ta...,friendship,Positive,"Billy, Timmy",


In [97]:
import pandas as pd
from google.colab import files

# Define the file name
file_name = 'your_dataframe.csv'

# Save the DataFrame to a CSV file in Colab's current working directory
df.to_csv(file_name, index=False)

# Create a link to download the CSV file
files.download(file_name)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>